# Вызов Си кода из питона
Основной инструмент для вызова кода, написанного на Си (или на С++, но с некоторыми ограничениями) - библиотека ctypes: https://docs.python.org/3/library/ctypes.html

In [1]:
import ctypes

## Загрузка динамической библиотеки
Для того, чтобы вызвать плюсовый код необходимо, чтобы он был обёрнут динамической библиотекой:
Пусть в square.cpp написана функция
```
int square(int a){
	return a * a;
}
```
И скомпилированна в динамическую библиотеку **libsquare.so**.
Функции, имеющиеся в динамической библиотеке можно посмотреть с помощью команды nm:
```
$ nm -D ./libsquare.so 
                 w __cxa_finalize@GLIBC_2.2.5
                 w __gmon_start__
                 w _ITM_deregisterTMCloneTable
                 w _ITM_registerTMCloneTable
00000000000010e9 T square
```

In [2]:
so_filename = './libsquare.so'

In [3]:
so_lib = ctypes.CDLL(so_filename)

После загрузки самой библиотеки, нужно выбрать функцию, которую вы планируете вызывать, а также указать её список аргументов *argtypes* и возвращаемое значение *restype*. Для этого необходимо использовать типы самого ctypes, в частности - c_int

In [4]:
c_square = so_lib.square
c_square.argtypes = [ctypes.c_int]
c_square.restype = ctypes.c_int

In [5]:
c_square(10)

100

Ниже представлены основные типы данных в ctypes:
| **ctypes type** |               **C type**               |      **Python type**     |
|:---------------:|:--------------------------------------:|:------------------------:|
| c_bool          | _Bool                                  | bool (1)                 |
| c_char          | char                                   | 1-character bytes object |
| c_wchar         | wchar_t                                | 1-character string       |
| c_byte          | char                                   | int                      |
| c_ubyte         | unsigned char                          | int                      |
| c_short         | short                                  | int                      |
| c_ushort        | unsigned short                         | int                      |
| c_int           | int                                    | int                      |
| c_uint          | unsigned int                           | int                      |
| c_long          | long                                   | int                      |
| c_ulong         | unsigned long                          | int                      |
| c_longlong      | __int64 or long long                   | int                      |
| c_ulonglong     | unsigned __int64 or unsigned long long | int                      |
| c_size_t        | size_t                                 | int                      |
| c_ssize_t       | ssize_t or Py_ssize_t                  | int                      |
| c_float         | float                                  | float                    |
| c_double        | double                                 | float                    |
| c_longdouble    | long double                            | float                    |
| c_char_p        | char* (NUL terminated)                 | bytes object or None     |
| c_wchar_p       | wchar_t* (NUL terminated)              | string or None           |
| c_void_p        | void*                                  | int or None              |

## Адреса как аргументы
Если функция получает на вход указатель, то в сtypes есть два способа использовать её:
*pointer()* и *byref()*. Первый создаёт обьект класса указатель, а второй создаёт ссылку (соответсвенно, второй работает быстрее)

In [6]:
libc = ctypes.cdll.LoadLibrary("libc.so.6")

In [7]:
i = ctypes.c_int()
f = ctypes.c_float()
s = ctypes.create_string_buffer(b'\000' * 32)
print(i.value, f.value, repr(s.value))

0 0.0 b''


In [8]:
libc.sscanf(b"1 3.14 Hello", b"%d %f %s", ctypes.byref(i), ctypes.byref(f), s)

3

In [9]:
print(i.value, f.value, repr(s.value))

1 3.140000104904175 b'Hello'


## Структуры
Если сишная функция использует какую-то структуру, то в коде питона её нужно продублировать с аргументом structure:

In [10]:
class POINT(ctypes.Structure):
    _fields_ = [("x", ctypes.c_double),
                ("y", ctypes.c_double),
                ("z", ctypes.c_double)]

имея в библиотеке функцию
```
double norm(const point p){
	return sqrt(p.x * p.x + p.y * p.y + p.z * p.z);
}
```

In [11]:
p = POINT(1, 2, 3)
p.x, p.y, p.z

(1.0, 2.0, 3.0)

In [12]:
norm = so_lib.norm
norm.argtypes = [POINT]
norm.restype = ctypes.c_double
norm(p)

3.7416573867739413

## Callback-функции
Иногда для работы кода необходимо иметь callback функцию, например, при использовании Си-шного qsort. Для этого можно воспользоваться CFUNCTYPE, первым который является оберткой над питоновской функцией, а получает первым аргументов возвращаемое значение и параметры - остальными.

In [13]:
IntArray5 = ctypes.c_int * 5
ia = IntArray5(5, 1, 7, 33, 99)

qsort = libc.qsort
qsort.restype = None

In [14]:
# Функция-фабрика для создания callback-функций для сортировки.
CMPFUNC = ctypes.CFUNCTYPE(ctypes.c_int, 
                           ctypes.POINTER(ctypes.c_int), 
                           ctypes.POINTER(ctypes.c_int))

In [15]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return 0
cmp_func = CMPFUNC(py_cmp_func)

In [16]:
qsort(ia, len(ia), ctypes.sizeof(ctypes.c_int), cmp_func) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 5 7
py_cmp_func 1 7


## ctypes и C++
Т.к в C++ имеется перезрузка функций, то построения их имен в единице трасляции становится менеее очевидными. Например функция *square*, приведенная выше, но написанная собранная как с++ программа, будет иметь вид:
```
00000000000011ca T _Z6squarei
```
Однако, если типы аргументов и возвращаемого значения Сишные, то вы все ещё можете вызвать её.
Также, вы можете погрузить функцию в окружение extern "C", которое указывает компилятору, что код, по возможности, надо собрать с помощью Си компилятора. Так например, можно обработать функции, имеющие ссылки в качестве аргумента (которые перейдут в указатели).
```
extern "C"{
    int square(int a){
        return a * a;
    }
}
```


# ctypes и numpy

In [17]:
import numpy as np

numpy умеет взаимодействовать с ctypes, в частности, создавать массивы с типом ctypes:

In [18]:
np.zeros(5, dtype=ctypes.c_double)

array([0., 0., 0., 0., 0.])

Также, к массивам numpy можно применять Сишные функции, имеющие в качестве аргумента указатель на массив. Делается это с помощью **data_as**

In [19]:
c_square_array = so_lib.square_array
c_square_array.argtypes = [ctypes.POINTER(ctypes.c_double), ctypes.c_int]
c_square_array.restype = None

In [20]:
a = np.arange(12).astype(np.float64)
a_ptr = a.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
a

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [21]:
c_square_array(a_ptr, a.shape[0])

In [22]:
a

array([  0.,   1.,   4.,   9.,  16.,  25.,  36.,  49.,  64.,  81., 100.,
       121.])

callback-функции создаются аналогично

In [23]:
c_square_array_ctypes = ctypes.CFUNCTYPE(None, ctypes.POINTER(ctypes.c_double), 
                                         ctypes.c_int)(c_square_array)

In [24]:
a = np.arange(12).astype(np.float64)
a_ptr = a.ctypes.data_as(ctypes.POINTER(ctypes.c_double))

c_square_array_ctypes = ctypes.CFUNCTYPE(None, ctypes.POINTER(ctypes.c_double), ctypes.c_int)(c_square_array)

c_square_array_ctypes(a_ptr, a.shape[0])
a

array([  0.,   1.,   4.,   9.,  16.,  25.,  36.,  49.,  64.,  81., 100.,
       121.])

# ctypes и numba

In [25]:
import numba

С помощью numba можно векторизовать callback функции из ctypes

In [26]:
c_square_ctypes = ctypes.CFUNCTYPE(ctypes.c_int, ctypes.c_int)(c_square)

In [27]:
@numba.vectorize('int32(int32)')
def example_func(x):
    return c_square_ctypes(x)

In [28]:
x = np.arange(9).astype(np.int32)

In [29]:
example_func(x)

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64], dtype=int32)

In [30]:
@numba.njit
def example_array_func(x):
    c_square_array_ctypes(x.ctypes, x.shape[0])

In [31]:
a

array([  0.,   1.,   4.,   9.,  16.,  25.,  36.,  49.,  64.,  81., 100.,
       121.])

In [32]:
example_array_func(a)

In [33]:
a

array([0.0000e+00, 1.0000e+00, 1.6000e+01, 8.1000e+01, 2.5600e+02,
       6.2500e+02, 1.2960e+03, 2.4010e+03, 4.0960e+03, 6.5610e+03,
       1.0000e+04, 1.4641e+04])

Однако при работе со слайсами и конвертации их в указатели нужно быть осторожными: длину шага слайса перевести в сишный указатель не получится

In [46]:
a = np.arange(12).astype(np.float64)
a

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [47]:
b = a[2::3]
b

array([ 2.,  5.,  8., 11.])

In [48]:
# example_array_func(b)
b_ptr = b.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
c_square_array_ctypes(b_ptr, b.shape[0])

In [49]:
a # поменялись b.shape[0] значений подряд

array([ 0.,  1.,  4.,  9., 16., 25.,  6.,  7.,  8.,  9., 10., 11.])

In [51]:
b # хотя slice b указывает не на подряд идущие значения

array([ 4., 25.,  8., 11.])